In [1]:
import numpy as np
import tensorflow as tf
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
from tensorflow.keras.datasets import mnist
(X_raw,_), (_,_)= mnist.load_data()   #Only using a train data in this case

In [3]:
n_img = X_raw.shape[0]
n_img

60000

In [4]:
epoch= 3000
n_batch =100

In [5]:
import numpy as np
X_re = X_raw.reshape(n_img,28,28,1)
scale_c = 255/2
X = (X_re - scale_c) / scale_c
real_1 = np.ones((n_batch,1))
fake_0 = np.zeros((n_batch,0))

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LeakyReLU

In [9]:
input_layer1 = Input(shape=(100,))
x1 = Dense(64*7*7)(input_layer1)
x1 = BatchNormalization()(x1)
x1 = Activation(LeakyReLU(0.3))(x1)
x1 = Reshape((7,7,64))(x1)
x1 = UpSampling2D()(x1)
x1 = Conv2D(32,kernel_size=(3,3),padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = Activation(LeakyReLU(0.3))(x1)
x1 = UpSampling2D()(x1)
output_layer1 = Conv2D(1,kernel_size=(3,3),padding='same',activation='tanh')(x1)
generator = Model(input_layer1,output_layer1)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              316736    
_________________________________________________________________
batch_normalization (BatchNo (None, 3136)              12544     
_________________________________________________________________
activation (Activation)      (None, 3136)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 32)        18464 

In [18]:
input_layer2 = Input(shape=(28,28,1))
x2 = Conv2D(64,kernel_size=(5,5),padding='same')(input_layer2)
x2 = Activation(LeakyReLU(0.3))(x2)
x2 = Dropout(0.25)(x2)
x2= Flatten()(x2)
output_layer2 = Dense(1,activation='sigmoid')(x2)
discriminator = Model(input_layer2, output_layer2)
discriminator.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
discriminator.trainable = False
discriminator.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 50177     
Total params: 51,841
Trainable params: 0
Non-trainable params: 51,841
_______________________________________________________

In [19]:
input_gan = Input(shape=(100,))
output_dis = discriminator(generator(input_gan))
gan = Model(input_gan,output_dis)
gan.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
gan.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
model (Functional)           (None, 28, 28, 1)         348161    
_________________________________________________________________
model_6 (Functional)         (None, 1)                 51841     
Total params: 400,002
Trainable params: 341,825
Non-trainable params: 58,177
_________________________________________________________________


In [21]:
loss_disc_real = [0]*epoch
loss_disc_fake = [0]*epoch
loss_disc_avg = [0]*epoch
loss_gan = [0]*epoch
acc_disc_real = [0]*epoch
acc_dic_fake = [0]*epoch
acc_disc_avg = [0]*epoch
acc_gan = [0]*epoch

for i in range(0,epoch):
    idx = np.random.randint(0,n_img,n_batch)
    imgs = X[idx]
    res_real = discriminator.train_on_batch(imgs,real_1)
    
    fake = np.random.normal(0,1,size=(n_batch,100))
    gen_imgs = generator.predict(fake)
    res_fake = discriminator.train_on_batch(gen_imgs,fake_0)
    
    loss_disc_avg_ith = np.add(res_real[0],res_fake[0])*0.5
    acc_disc_avg_ith = np.add(res_real[1],res_fake[1])*0.5
    
    res_gan = gan.train_on_batch(fake,real_1)
    
    loss_disc_real[i] = res_real[0]
    loss_disc_fake[i] = res_fake[0]
    loss_disc_avg[i] = loss_disc_avg_ith
    loss_gan[i] = res_gan[0]
    
    acc_disc_real[i] = res_real[1]
    acc_disc_fake[i] = res_fake[1]
    acc_disc_avg[i] = acc_disc_avg_ith
    acc_gan[i] = res_gan[1]
    
    print('epoch: %d'%i,
         '판별손실평균: %.4f'%loss_disc_avg_ith,
         '판별정확도평균: %.4f'%acc_disc_avg_ith,
         '생성손실: %.4f'%res_gan[0],
         '생성정확도: %.4f'%res_gan[1])

ValueError: in user code:

    C:\Users\ss263\Desktop\AC\envs\tf24\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\ss263\Desktop\AC\envs\tf24\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ss263\Desktop\AC\envs\tf24\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ss263\Desktop\AC\envs\tf24\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ss263\Desktop\AC\envs\tf24\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ss263\Desktop\AC\envs\tf24\lib\site-pack